In [27]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, roc_curve, auc

data = pd.read_csv('diabetes.csv')
X = data.drop('Outcome', axis=1).values
y = data['Outcome'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

class Perceptron:
    def __init__(self, learning_rate=0.01, n_iters=1000):
        self.learning_rate = learning_rate
        self.n_iters = n_iters
        self.weights = None
        self.bias = None
        self.accuracy_history = []

    def activation_function(self, z):
        return np.where(z >= 0, 1, 0)
    
    def fit(self, X, y): 
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0
        for _ in range(self.n_iters):
            for i, x_i in enumerate(X):
                linear_output = np.dot(x_i, self.weights) + self.bias
                y_predicted = self.activation_function(linear_output)
                update = self.learning_rate * (y[i] - y_predicted)
                self.weights += update * x_i
                self.bias += update
            accuracy = np.mean(self.predict(X) == y)
            self.accuracy_history.append(accuracy)

    def predict(self, X):
        linear_output = np.dot(X, self.weights) + self.bias
        y_predicted = self.activation_function(linear_output)
        return y_predicted
start_time = time.time()
perceptron = Perceptron(learning_rate=0.01, n_iters=1000)
perceptron.fit(X_train_scaled, y_train)
end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time= {execution_time:.2f} seconds")
y_pred = perceptron.predict(X_test_scaled)
accuracy = np.mean(y_pred == y_test)
print(f"Accuracy= {accuracy * 100:.2f}%")

true_p = np.sum((y_test == 1) & (y_pred == 1))  
true_n = np.sum((y_test == 0) & (y_pred == 0))
false_p = np.sum((y_test == 0) & (y_pred == 1)) 
false_n = np.sum((y_test == 1) & (y_pred == 0))  
precision = true_p / (true_p + false_p) if (true_p + false_p) > 0 else 0
recall = true_p / (true_p + false_n) if (true_p + false_n) > 0 else 0

print(f"Precision= {precision:.2f}")
print(f"Recall= {recall:.2f}")
confm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(confm, annot=True, fmt='d', cmap='Blues', xticklabels=['Non-Diabetic', 'Diabetic'], yticklabels=['Non-Diabetic', 'Diabetic'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.savefig('confusion_matrix.png') 
plt.close() 
fpr, tpr, thresholds = roc_curve(y_test, y_pred) 
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='red', linestyle='--') 
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.savefig('roc_curve.png') 
plt.close() 

features = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']
importance = np.abs(perceptron.weights)

plt.figure(figsize=(10, 6))
plt.barh(features, importance, color='skyblue')
plt.xlabel('Absolute Weight Value')
plt.title('Feature Importance in Perceptron Model')
plt.savefig('feature_importance.png') 
plt.close()  


Execution time= 13.56 seconds
Accuracy= 76.62%
Precision= 0.73
Recall= 0.55
